In [ ]:
import tensorflow as tf
import numpy      as np

import pathlib
import data
import os

from tensorflow.keras.models import load_model

In [ ]:
AUTOTUNE   = tf.data.experimental.AUTOTUNE
dataObj    = data.Data()
numClasses = dataObj.numClasses
classNames = np.array(dataObj.classes)

In [ ]:
def getBestSavedModel(modelCheckpointPath):
    dirPaths  = modelCheckpointPath.glob("*")
    minLoss   = float('inf')
    bestModel = ""
    for dirPath in dirPaths:
        modelname = pathlib.Path(dirPath.name).stem
        loss = float(modelname.split("_")[-1])
        if loss <= minLoss:
            bestModel = dirPath
            minLoss   = loss
    return str(bestModel)

In [ ]:
def evaluateCNN():
    def getDataSet(dataDirectory):
        imagePathsDataset = tf.data.Dataset.list_files(str(dataDirectory/'*/*'))
        return imagePathsDataset
    
    def getLabeledData(imagePath):
        def getLabel(imagePath):
            # convert the path to a list of path components
            parts = tf.strings.split(imagePath, os.path.sep)
            # The second to last is the class-directory
            return parts[-2] == classNames
    
        def decodeImg(img):
            # convert the compressed string to a 3D uint8 tensor
            img = tf.image.decode_jpeg(img, channels=3)
            # Use `convert_image_dtype` to convert to floats in the [0,1] range.
            img = tf.image.convert_image_dtype(img, tf.float32)
            # resize the image to the desired size.
            return tf.image.resize(img, [299, 299])
    
        label = getLabel(imagePath)
        # load the raw data from the file as a string
        img = tf.io.read_file(imagePath)
        img = decodeImg(img)
        return img, label
    
    BATCH_SIZE = 32
    rootPath            = pathlib.Path(r"/home/jupyter/action-recognition/")
    testDataDirectory   = rootPath/'Frames'/'Test'
    modelCheckpointPath = rootPath/'Callbacks/CNN'/f'{numClasses}'/'ModelCheckpoint'

    testDataset = getDataSet(testDataDirectory)
    testDataset = testDataset.map(getLabeledData, num_parallel_calls = AUTOTUNE)
    testDataset = testDataset.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)

    savedModelPath = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    
    savedModel = load_model(savedModelPath)
    results = savedModel.evaluate(testDataset)
    return results

results = evaluateCNN()
print("Loss: ", results[0], "Accuracy: ", results[1])

In [ ]:
def evaluateRNN():
    def getDataset(sequences, labels):
        dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
        return dataset
    
    def getSequences(datasetType):
        maxFrameCount = dataObj.getMaxFrameCount()
        sequences = []
        labels    = []
        sequencesPath = pathlib.Path(r"/home/jupyter/action-recognition/Sequences")
        for dataRow in dataObj.data[:]:
            if(datasetType == dataRow[0]):
                sequencePath = sequencesPath/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence.npy")
                sequence     = np.load(sequencePath)
                sequence     = np.pad(sequence, ((0, maxFrameCount - int(dataRow[3])), (0, 0)), 'edge')
                sequences.append(sequence)
                label = dataObj.getClassIndex(dataRow[1])
                labels.append(label)
        return np.array(sequences), np.array(labels, dtype=np.uint8)
    
    BATCH_SIZE = 32
    rootPath = pathlib.Path(r"/home/jupyter/action-recognition/")
    modelCheckpointPath = rootPath/'Callbacks/RNN'/f'{numClasses}'/'ModelCheckpoint'
    
    testSequences, testLabels = getSequences('Test')
    testDataset = getDataset(testSequences, testLabels)
    testDataset = testDataset.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)
    
    savedModelPath = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    
    savedModel = load_model(savedModelPath)
    results = savedModel.evaluate(testDataset)
    return results

results = evaluateRNN()
print("Loss: ", results[0], "Accuracy: ", results[1])